In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [6]:
#Load diabetes dataset
diabetes_df = pd.read_csv("../week_13/diabetes.csv")
diabetes_df

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                       0.351   31        0  
2                       0.672   32        1  
3                       0.167   21        0  
4                       2.288   33        1  
..                        ...  ...      ...  
763                     0.171   63        0  
764                     0.340   27        0  
765                     0.245   30        0  
766                     0.349   47        1  
767                     0.315   23        0  

[768 rows x 9 columns]

In [7]:
X = diabetes_df.drop('Outcome',axis=1)
y = diabetes_df['Outcome']


In [8]:
#SMOTE-ENN
from collections import Counter
from imblearn.combine import SMOTEENN 
from imblearn.under_sampling import EditedNearestNeighbours

print('Original dataset shape %s' % Counter(y))
#sme = SMOTEENN(random_state=42)
sme = SMOTEENN(random_state=42,enn=EditedNearestNeighbours(sampling_strategy='all'))
X_res, y_res = sme.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

print(X_res.shape,y_res.shape)
print(X.shape,y.shape)


Original dataset shape Counter({0: 500, 1: 268})
Resampled dataset shape Counter({1: 298, 0: 229})
(527, 8) (527,)
(768, 8) (768,)


In [9]:
#Split into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size = 0.25,random_state=42)

#Perform standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [10]:
#DecisionTreeModel
from sklearn import tree
dtmodel = tree.DecisionTreeClassifier(max_depth = 8,random_state=42)
dtmodel = dtmodel.fit(X_train,y_train)
y_pred = dtmodel.predict(X_test)

from sklearn.metrics import confusion_matrix,plot_confusion_matrix,classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90        61
           1       0.89      0.94      0.92        71

    accuracy                           0.91       132
   macro avg       0.91      0.91      0.91       132
weighted avg       0.91      0.91      0.91       132



In [11]:
#Perform logistic regression on combined sampled data
from sklearn.linear_model import LogisticRegression
#Logistic regression
lregmodel = LogisticRegression(random_state=42)
lregmodel.fit(X_train,y_train)
y_pred = dtmodel.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90        61
           1       0.89      0.94      0.92        71

    accuracy                           0.91       132
   macro avg       0.91      0.91      0.91       132
weighted avg       0.91      0.91      0.91       132

